<a href="https://colab.research.google.com/github/jmgang/SpoTwoFy-project-notebooks/blob/main/notebooks/1_SpoTwoFy_Playlist_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spotipy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import ast
import math
import getpass
import time
import os

from tqdm import tqdm

In [3]:
# Mount GDrive folders
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Set home directory
import os
home_dir = "/content/drive/MyDrive/Colab Notebooks/Sprint 3/"
os.chdir(home_dir)

In [5]:
# Make data/playlist/ directory
os.makedirs('data/playlists', exist_ok=True)

In [11]:
client_id = 'ecf55d3b31964b53af6841e39dfebcb2'

In [12]:
client_secret = '73eb4ae7d6774f0e8d67ec2da2d9e5a8'

In [14]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                      client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [17]:
def get_playlist_ids_names(KEYWORD):
  playlist_ids = []
  playlist_names = []

  N = 100
  # Get playlist in batches of 50
  for n in np.arange(N//50):
      offset= 50*n
      print("Getting batch %d of search results for keyword: %s ..." % (n,KEYWORD), end='' )
      results = sp.search(q=KEYWORD, type='playlist' , market='PH', offset = offset, limit=50)
      playlist_ids.extend([p['href'].split('/')[5] for p in results['playlists']['items']])
      playlist_names.extend([p['name'] for p in results['playlists']['items']])
      print("  DONE!")
  return playlist_ids, playlist_names

In [18]:
def get_track_audio_features_data(track_ids):
    audio_features_data = []
    audio_features_keys = ['danceability','energy','key','loudness','mode',\
                            'speechiness','acousticness','instrumentalness','liveness',\
                            'valence','tempo','duration_ms']

    total_iterations = math.ceil(len(track_ids) / 100)

    for i in tqdm(range(0, len(track_ids), 100), total=total_iterations, desc='Fetching audio features data '):
        track_ids_chunk = track_ids[i:i + 100]
        track_audio_features = sp.audio_features(track_ids_chunk)

        for audio_feature in track_audio_features:
          if audio_feature is not None:
            audio_feature_data = dict()
            audio_feature_data['track_id'] = audio_feature['id'] if 'id' in audio_feature.keys() else 'none'
            for key in audio_features_keys:
              try:
                  audio_feature_data[key] = audio_feature[key]
              except:
                  audio_feature_data[key] = None
            audio_features_data.append(audio_feature_data)

        time.sleep(3)
    return audio_features_data

In [19]:
def get_playlist_tracks_data(playlist_info):
  playlist_track_data = []
  if 'tracks' not in playlist_info or playlist_info['tracks'] is None:
        print(f"No tracks data for playlist id: {playlist_info.get('playlist_id', 'Unknown')}")
        return []
  for track_data in playlist_info['tracks']:
      if 'track' not in track_data or track_data['track'] is None:
        continue
      relevant_track_data = { key: track_data['track'][key] for key in ['name','popularity','duration_ms'] }
      relevant_track_data['track_id'] = track_data['track']['id']
      relevant_track_data['artist_id'] = [artist['id'] for artist in track_data['track']['artists'] ]
      relevant_track_data['artist_name'] = [artist['name'] for artist in track_data['track']['artists'] ]
      relevant_track_data['num_artists'] = len([artist['id'] for artist in track_data['track']['artists']])
      # If single artist track, convert list to single-element
      relevant_track_data['artist_id'] = relevant_track_data['artist_id'][0] if len(relevant_track_data['artist_id'])==1 \
                                          else relevant_track_data['artist_id']
      relevant_track_data['artist_name'] = relevant_track_data['artist_name'][0] if len(relevant_track_data['artist_name'])==1 \
                                          else relevant_track_data['artist_name']
      relevant_track_data['album_id'] = track_data['track']['album']['uri'].split(":")[2]
      relevant_track_data['release_date'] = track_data['track']['album']['release_date']
      relevant_track_data['playlist_id'] = playlist_info['playlist_id']
      relevant_track_data['playlist_name'] = playlist_info['playlist_name']
      playlist_track_data.append(relevant_track_data)
  return playlist_track_data

In [20]:
# Helper function to get playlist data in dict format
def get_playlist_data(playlist_id):
    playlist_data = sp.playlist(playlist_id)
    if playlist_data is None:
        print(f"Failed to fetch data for playlist id: {playlist_id}")
        return None
    track_data = []
    relevant_playlist_data = {'playlist_id': playlist_id}
    relevant_playlist_data['playlist_name'] = playlist_data['name']
    relevant_playlist_data['playlist_total_tracks'] = playlist_data['tracks']['total']
    relevant_playlist_data['owner_id'] = playlist_data['owner']['id']
    relevant_playlist_data['owner_name'] = playlist_data['owner']['display_name']
    relevant_playlist_data['total_followers'] = playlist_data['followers']['total']
    tracks = playlist_data['tracks']
    track_data.extend(tracks['items'])

    # Tracks might contain additional items
    while tracks['next']:
        tracks = sp.next(tracks)
        track_data.extend(tracks['items'])
        time.sleep(0.5)

    relevant_playlist_data['tracks'] = track_data

    return relevant_playlist_data

### 1. Read playlist tracks of 2 genres

#### Genre 1: EDM

In [21]:
#set keyword
KEYWORD1='sad'

# Get playlists of searched keyword

In [22]:
playlist_ids, playlist_names = get_playlist_ids_names(KEYWORD1)

Getting batch 0 of search results for keyword: sad ...  DONE!
Getting batch 1 of search results for keyword: sad ...  DONE!


# Get playlist data with track information

In [23]:
playlist_data_list = []
playlist_track_information = []
progress_bar = tqdm(enumerate(playlist_ids), total=len(playlist_ids), desc='Fetching playlist data ')
for i, playlist_id in progress_bar:
    try:
        relevant_playlist_data = get_playlist_data(playlist_id)
        if relevant_playlist_data:
          playlist_data_list.append(relevant_playlist_data)
          playlist_track_information.extend(get_playlist_tracks_data(relevant_playlist_data))
        time.sleep(1)
    except Exception as e:
        # Print track id and error msg
        print(f'Error requesting data for playlist id {playlist_id}: {e}')
        continue

Fetching playlist data : 100%|██████████| 100/100 [02:58<00:00,  1.78s/it]


## Saving Playlist data

In [24]:
playlist_data_df = pd.DataFrame(playlist_data_list)
playlist_data_df.drop('tracks', inplace=True, axis=1)
playlist_data_df = playlist_data_df.sort_values('total_followers',ascending=False)
playlist_data_df  = playlist_data_df[playlist_data_df['playlist_name'].str.lower().str.contains(KEYWORD1.lower())]
playlist_data_df.head()

,playlist_id,playlist_name,playlist_total_tracks,owner_id,owner_name,total_followers
12,37i9dQZF1DX7qK8ma5wgG1,Sad Songs,80,spotify,Spotify,1747337
14,37i9dQZF1DWVV27DiNWxkR,Sad Indie,108,spotify,Spotify,1646905
2,37i9dQZF1DWSqBruwoIXkA,sad hour,150,spotify,Spotify,1646608
22,37i9dQZF1DWW2hj3ZtMbuO,sad girl starter pack,75,spotify,Spotify,1447776
0,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music,133,yj9ghsz7fmkg4nu9lkx02wute,Lofish Records ☄️,975886


In [25]:
filename = "data/playlists/project_playlists/"+KEYWORD1+"_playlist_data.csv"
playlist_data_df.to_csv(filename,encoding='utf=8',index=False)

## Saving Playlist tracks data

In [26]:
playlist_track_data_df = pd.DataFrame(playlist_track_information)
playlist_track_data_df = playlist_track_data_df.drop_duplicates(subset='track_id').reset_index(drop=True)
playlist_track_data_df  = playlist_track_data_df[playlist_track_data_df['playlist_name'].str.lower().str.contains(KEYWORD1.lower())]
playlist_track_data_df = playlist_track_data_df.reset_index(drop=True)
playlist_track_data_df

,name,popularity,duration_ms,track_id,artist_id,artist_name,num_artists,album_id,release_date,playlist_id,playlist_name
0,Wish You The Best,35,210880,3W5KHP0Yr56ejVDns9vci3,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi,1,7dhkqX7ovYADB1xwZr9bwS,2023-04-14,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
1,Glimpse of Us,85,233456,6xGruZOHLs39ZbVccQTuPZ,3MZsBdqDrRTJihTHQrO6Dq,Joji,1,6ZZvx0aefZV3LKa053fn71,2022-06-10,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
2,Daylight,95,212953,1odExI7RdWc4BT515LTAwj,33NVpKoXjItPwUJTMZIOiY,David Kushner,1,6NcI39WPu4kY6Tul11nhSv,2023-04-14,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
3,Before You Go,86,215106,2gMXnyrvIjhVBUZwvLZDMP,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi,1,2wiPF3m0ylst0JSk1IvZL8,2019-11-22,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
4,the grudge,88,189386,3Nl5OkkmS5DaBZvuYofpAt,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,1,1xJHno7SmdVtZAtXbdbDZp,2023-09-08,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
...,...,...,...,...,...,...,...,...,...,...,...
6675,Me Deixe Sumir,38,217668,2DHidHdCswIjNE4kgTK2fX,7wYvRV1aFfU9UlGDxoZQFY,Hiosaki,1,17lHG7WmEb33MDCEENNZar,2020-04-03,37i9dQZF1DX0H97kz3DdJ6,sad sla
6676,deitado no quarto,30,176083,7wBfwQ1dLrf3QiQV7WRN6O,"[1iffB8upqqDnx6UX8P3kz2, 0GVaM9LtM51CEGq6FJ3MC8]","[Sobral, Zant]",2,5AxkAvYMhR5pUR2XRpgWFs,2019-10-28,37i9dQZF1DX0H97kz3DdJ6,sad sla
6677,Ainda Lembro,37,146456,3RkTmbR7D01mq46FNp1mI0,"[4OFUU6MkPNc2X96UJMlR0h, 1iffB8upqqDnx6UX8P3kz2]","[Sadstation, Sobral]",2,2tG0W9n2wjwG8x0eA6LpZY,2019-07-01,37i9dQZF1DX0H97kz3DdJ6,sad sla
6678,Que não te falte amor,46,163687,4e8mbXLx3DRgLbTafPpNat,"[6LRVpC7lJ4IYrPNeAiylYB, 6Ae0wz09vBFYZXJ2bJAKUl]","[Lil Chainz, Kiaz]",2,70YMa2rpfCE4LQ1Q7clQws,2019-01-12,37i9dQZF1DX0H97kz3DdJ6,sad sla


In [27]:
filename = "data/playlists/project_playlists/"+KEYWORD1+"_playlist_tracks.csv"
playlist_track_data_df.to_csv(filename,encoding='utf=8',index=False)

# Getting Audio Features of Tracks

In [28]:
track_ids = playlist_track_data_df.track_id.unique().tolist()
len(track_ids)

6680

In [29]:
track_audio_features = get_track_audio_features_data(track_ids)
len(track_audio_features)

Fetching audio features data : 100%|██████████| 67/67 [03:36<00:00,  3.23s/it]


6680

In [30]:
track_audio_features_df = pd.DataFrame(track_audio_features)
track_audio_features_df

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,3W5KHP0Yr56ejVDns9vci3,0.678,0.480,3,-3.607,1,0.0298,0.588,0.000000,0.0954,0.490,107.934,210880
1,6xGruZOHLs39ZbVccQTuPZ,0.440,0.317,8,-9.258,1,0.0531,0.891,0.000005,0.1410,0.268,169.914,233456
2,1odExI7RdWc4BT515LTAwj,0.508,0.430,2,-9.475,0,0.0335,0.830,0.000441,0.0930,0.324,130.090,212954
3,2gMXnyrvIjhVBUZwvLZDMP,0.459,0.575,3,-4.858,1,0.0573,0.604,0.000000,0.0885,0.183,111.881,215107
4,3Nl5OkkmS5DaBZvuYofpAt,0.548,0.307,10,-9.214,1,0.0781,0.916,0.000000,0.0828,0.317,127.923,189386
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6675,2DHidHdCswIjNE4kgTK2fX,0.773,0.421,4,-8.212,0,0.0870,0.654,0.000011,0.0845,0.339,125.944,217669
6676,7wBfwQ1dLrf3QiQV7WRN6O,0.531,0.490,4,-9.190,1,0.1050,0.699,0.000017,0.1430,0.462,119.757,176083
6677,3RkTmbR7D01mq46FNp1mI0,0.546,0.183,5,-26.733,1,0.1630,0.834,0.000004,0.0927,0.242,165.789,146457
6678,4e8mbXLx3DRgLbTafPpNat,0.464,0.247,6,-13.423,0,0.0470,0.823,0.000000,0.1430,0.503,94.842,163687


In [31]:
filename = "data/playlists/project_playlists/"+KEYWORD1+"_tracks_audio_features.csv"
track_audio_features_df.to_csv(filename,encoding='utf=8',index=False)

In [32]:
overlapping_columns = [col for col in track_audio_features_df.columns if col in playlist_track_data_df.columns and col != 'track_id']
track_audio_features_df.drop(columns=overlapping_columns, inplace=True)
overall_playlist_track_data_df = track_audio_features_df.merge(playlist_track_data_df, on='track_id')
overall_playlist_track_data_df.rename(columns={'name' : 'track_name'}, inplace=True)
overall_playlist_track_data_df

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,track_name,popularity,duration_ms,artist_id,artist_name,num_artists,album_id,release_date,playlist_id,playlist_name
0,3W5KHP0Yr56ejVDns9vci3,0.678,0.480,3,-3.607,1,0.0298,0.588,0.000000,0.0954,...,Wish You The Best,35,210880,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi,1,7dhkqX7ovYADB1xwZr9bwS,2023-04-14,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
1,6xGruZOHLs39ZbVccQTuPZ,0.440,0.317,8,-9.258,1,0.0531,0.891,0.000005,0.1410,...,Glimpse of Us,85,233456,3MZsBdqDrRTJihTHQrO6Dq,Joji,1,6ZZvx0aefZV3LKa053fn71,2022-06-10,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
2,1odExI7RdWc4BT515LTAwj,0.508,0.430,2,-9.475,0,0.0335,0.830,0.000441,0.0930,...,Daylight,95,212953,33NVpKoXjItPwUJTMZIOiY,David Kushner,1,6NcI39WPu4kY6Tul11nhSv,2023-04-14,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
3,2gMXnyrvIjhVBUZwvLZDMP,0.459,0.575,3,-4.858,1,0.0573,0.604,0.000000,0.0885,...,Before You Go,86,215106,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi,1,2wiPF3m0ylst0JSk1IvZL8,2019-11-22,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
4,3Nl5OkkmS5DaBZvuYofpAt,0.548,0.307,10,-9.214,1,0.0781,0.916,0.000000,0.0828,...,the grudge,88,189386,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,1,1xJHno7SmdVtZAtXbdbDZp,2023-09-08,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6675,2DHidHdCswIjNE4kgTK2fX,0.773,0.421,4,-8.212,0,0.0870,0.654,0.000011,0.0845,...,Me Deixe Sumir,38,217668,7wYvRV1aFfU9UlGDxoZQFY,Hiosaki,1,17lHG7WmEb33MDCEENNZar,2020-04-03,37i9dQZF1DX0H97kz3DdJ6,sad sla
6676,7wBfwQ1dLrf3QiQV7WRN6O,0.531,0.490,4,-9.190,1,0.1050,0.699,0.000017,0.1430,...,deitado no quarto,30,176083,"[1iffB8upqqDnx6UX8P3kz2, 0GVaM9LtM51CEGq6FJ3MC8]","[Sobral, Zant]",2,5AxkAvYMhR5pUR2XRpgWFs,2019-10-28,37i9dQZF1DX0H97kz3DdJ6,sad sla
6677,3RkTmbR7D01mq46FNp1mI0,0.546,0.183,5,-26.733,1,0.1630,0.834,0.000004,0.0927,...,Ainda Lembro,37,146456,"[4OFUU6MkPNc2X96UJMlR0h, 1iffB8upqqDnx6UX8P3kz2]","[Sadstation, Sobral]",2,2tG0W9n2wjwG8x0eA6LpZY,2019-07-01,37i9dQZF1DX0H97kz3DdJ6,sad sla
6678,4e8mbXLx3DRgLbTafPpNat,0.464,0.247,6,-13.423,0,0.0470,0.823,0.000000,0.1430,...,Que não te falte amor,46,163687,"[6LRVpC7lJ4IYrPNeAiylYB, 6Ae0wz09vBFYZXJ2bJAKUl]","[Lil Chainz, Kiaz]",2,70YMa2rpfCE4LQ1Q7clQws,2019-01-12,37i9dQZF1DX0H97kz3DdJ6,sad sla


In [33]:

overall_playlist_track_data_df['duration_mins'] = overall_playlist_track_data_df['duration_ms'] / 60000
#tag genre with keyword
overall_playlist_track_data_df['genre'] = KEYWORD1
overall_playlist_track_data_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,duration_ms,artist_id,artist_name,num_artists,album_id,release_date,playlist_id,playlist_name,duration_mins,genre
0,3W5KHP0Yr56ejVDns9vci3,0.678,0.480,3,-3.607,1,0.0298,0.588,0.000000,0.0954,...,210880,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi,1,7dhkqX7ovYADB1xwZr9bwS,2023-04-14,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music,3.514667,sad
1,6xGruZOHLs39ZbVccQTuPZ,0.440,0.317,8,-9.258,1,0.0531,0.891,0.000005,0.1410,...,233456,3MZsBdqDrRTJihTHQrO6Dq,Joji,1,6ZZvx0aefZV3LKa053fn71,2022-06-10,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music,3.890933,sad
2,1odExI7RdWc4BT515LTAwj,0.508,0.430,2,-9.475,0,0.0335,0.830,0.000441,0.0930,...,212953,33NVpKoXjItPwUJTMZIOiY,David Kushner,1,6NcI39WPu4kY6Tul11nhSv,2023-04-14,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music,3.549217,sad
3,2gMXnyrvIjhVBUZwvLZDMP,0.459,0.575,3,-4.858,1,0.0573,0.604,0.000000,0.0885,...,215106,4GNC7GD6oZMSxPGyXy4MNB,Lewis Capaldi,1,2wiPF3m0ylst0JSk1IvZL8,2019-11-22,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music,3.585100,sad
4,3Nl5OkkmS5DaBZvuYofpAt,0.548,0.307,10,-9.214,1,0.0781,0.916,0.000000,0.0828,...,189386,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,1,1xJHno7SmdVtZAtXbdbDZp,2023-09-08,3c0Nv5CY6TIaRszlTZbUFk,sad songs 2023 😢 crying and depressing music,3.156433,sad


In [34]:
overall_playlist_track_data_df.to_csv("data/playlists/project_playlists/"+KEYWORD1+"_playlist_tracks_data.csv", index=False, encoding='utf-8')